In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

In [7]:
# 1. 도구를 정의합니다

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

In [9]:
### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

In [11]:
# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [12]:
# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

In [14]:
# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings(model="text-embedding-3-small"))

In [15]:
# Retriever를 생성합니다.
retriever = vector.as_retriever()

In [16]:
# create_retriever_tool() 로 retriever를 도구로 변환합니다.
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",                                                        # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [17]:
### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

In [18]:
# 2. LLM 을 정의합니다 

# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [19]:
# 3. Prompt 를 정의합니다 

# Prompt 를 정의합니다
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [21]:
# 4. Agent 를 정의합니다 

# 에이전트를 생성합니다.
agent = create_tool_calling_agent(llm, tools, prompt)     # llm, tools, prompt를 인자로 사용합니다.

In [23]:
# 5. AgentExecutor 를 정의합니다 

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [24]:
#6. 채팅 기록을 수행하는 메모리를 추가합니다. 

store = {}                                          # session_id 를 저장할 딕셔너리 생성

def get_session_history(session_ids):               # session_id 를 기반으로 세션 기록을 가져오는 함수
    if session_ids not in store:                    # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장

    return store[session_ids]                       # 해당 세션 ID에 대한 세션 기록 반환

In [25]:
# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,    
    get_session_history,                    # 대화 session_id    
    input_messages_key="input",             # 프롬프트의 질문이 입력되는 key: "input"                                
    history_messages_key="chat_history",    # 프롬프트의 메시지가 입력되는 key: "chat_history"
)

In [26]:
#7. Agent 파서를 정의합니다.

agent_stream_parser = AgentStreamParser()

In [28]:
#8. 에이전트를 실행하고 결과를 확인합니다.

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(      
    {"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘"},
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 구글 앤스로픽 투자 금액
Log: 
Invoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`



[관찰 내용]
Observation: 1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 
앤스로픽 및 오픈AI와 협력을 확대하는 추세
KEY Contents
£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 
4년간 30억 달러 규모의 계약을 체결
∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) 
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) 
클라우드 플랫폼을 사용해 AI 모델을 훈련
£ 

In [29]:
#8. 에이전트를 실행하고 결과를 확인합니다.

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(      
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Google has invested a total of up to $2 billion in Anthropic, with an initial investment of $500 million. The remaining $1.5 billion is planned for future investment.


In [30]:
#8. 에이전트를 실행하고 결과를 확인합니다.

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(      
    {"input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"},
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://m.blog.naver.com/hoga1201/223571117829', 'content': '2024년 프로야구 포스트시즌 가을야구 플레이오프 진출팀 예상 분석 상대팀 별 잔여경기 : 네이버 블로그 2024년 프로야구 포스트시즌 가을야구 플레이오프 진출팀 예상 분석 상대팀 별 잔여경기 그래서 오늘은 KBO 남은 상대팀 경기 수와 우승 및 와일드카드 진출 가능성에 대한 경우의 수에 대해서 분석해 보도록 하겠습니다. 모든 팀이 가을야구에 대한 가능성이 남아있는 상황이라서, 남은 한 경기한 경기가 정말 피 말리는 승부가 될 것 같습니다. - KIA의 매직넘버는 현재 2등 삼성이 남은 17경기를 전승을 해도 11승 7패만 하면 자력 우승이 가능해질 것 같습니다. 위 순위표에서 각 구단별로 거둔 승수와 남은 경기 승패율 비교해서 내 팀이 가을야구 진출하려면 몇 승 이상을 해야 하는지 따져보면 될 것 같습니다. {"title":"2024년 프로야구 포스트시즌 가을야구 플레이오프 진출팀 예상 분석 상대팀 별 잔여경기","source":"https://blog.naver.com/hoga1201/223571117829","blogName":"호가의 일..","blogId":"god1986","domainIdOrBlogId":"hoga1201","nicknameOrBlogId":"호가","logNo":223571117829,"smartEditorVersion":4,"lineDisplay":true,"outsideDisplay":false,"meDisplay":true,"cafeDisplay":true,"blogDisplay":

In [31]:
#8. 에이전트를 실행하고 결과를 확인합니다.

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(      
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
2024년 프로야구 플레이오프 진출 팀은 기아, 삼성, LG, 두산, KT입니다! 치열한 경쟁 속에서 어떤 팀이 우승할지 기대됩니다! ⚾️🔥 #프로야구 #플레이오프


In [32]:
#8. 에이전트를 실행하고 결과를 확인합니다.

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(      
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024 한국 시리즈 일정
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024 한국 시리즈 일정'}`



[관찰 내용]
Observation: [{'url': 'https://m.blog.naver.com/baekhw1/223600397138?isInf=true', 'content': '2024 KBO 프로야구 한국시리즈 일정 총정리!(방식 구장 우승팀 뜻 우승상금 룰 경기수) : 네이버 블로그 2024 KBO 프로야구 한국시리즈 일정 총정리!(방식 구장 우승팀 뜻 우승상금 룰 경기수) 2024 KBO 프로야구 포스트시즌 한국시리즈는 10월 21일부터 29일로 10월 13일부터 19일까지 진행된다. 승리한 팀이 3위 LG트윈스와 준플레이오프(5전 3선승제)를 치룬다. 여기서 승리한 팀이 2위 삼성 라이온즈와 플레이오프(5전 3선승제)를 치룰 예정이다. 1차전 10월 21일 월요일 18시 30분 2차전 10월 22일 화요일 18시 30분 3차전 10월 24일 목요일 18시 30분 4차전 10월 25일 금요일 18시 30분 5차전 10월 27일 일요일 14시 6차전 10월 28일 월요일 18시 30분 7차전 10월 29일 화요일 18시 30분 한국시리즈 경기수 룰 한국시리즈 우승팀 우승상금 한국시리즈 일정 총정리!(방식 구장 #한국시리즈 {"title":"2024 KBO 프로야구 한국시리즈 일정 총정리!(방식 구장 우승팀 뜻 우승상금 룰 경기수)","source":"https://blog.naver.com/baekhw1/223600397138","blogName":"빠게룽","blogId":"baekhw1","domainIdOrBlogId":"baekhw1","nicknameOrBlogId":"빠게룽","logNo":223600397138,"smartEditorVersion":4,"meDisplay":true,